## Data sources
https://www.freecodecamp.org/news/how-to-build-your-first-neural-network-to-predict-house-prices-with-keras-f8db83049159/

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
df = pd.read_csv('housepricedata.csv')

In [4]:
df

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
0,8450,7,5,856,2,1,3,8,0,548,1
1,9600,6,8,1262,2,0,3,6,1,460,1
2,11250,7,5,920,2,1,3,6,1,608,1
3,9550,7,5,756,1,0,3,7,1,642,0
4,14260,8,5,1145,2,1,4,9,1,836,1
...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,5,953,2,1,3,7,1,460,1
1456,13175,6,6,1542,2,0,3,7,2,500,1
1457,9042,7,9,1152,2,0,4,9,2,252,1
1458,9717,5,6,1078,1,0,2,5,0,240,0


## DataSet
#### X features and y output

In [5]:
X = df.values[:,:-1]
Y = df.values[:,-1]
print(X)
print(Y)

[[ 8450     7     5 ...     8     0   548]
 [ 9600     6     8 ...     6     1   460]
 [11250     7     5 ...     6     1   608]
 ...
 [ 9042     7     9 ...     9     2   252]
 [ 9717     5     6 ...     5     0   240]
 [ 9937     5     6 ...     6     0   276]]
[1 1 1 ... 1 0 0]


### Normalization

In [6]:
print("Before normalization:")
for i in range(X.shape[1]):
    print(f"{df.columns[i]}, Min pre normalization: {np.max(X[:,i]):0.2f}, {np.min(X[:,i]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)

print("\nAfter normalization:")
for i in range(X.shape[1]):
    print(f"{df.columns[i]}, Min post normalization: {np.max(Xn[:,i]):0.2f}, {np.min(Xn[:,i]):0.2f}")  #df.columns --> get columns name

Before normalization:
LotArea, Min pre normalization: 215245.00, 1300.00
OverallQual, Min pre normalization: 10.00, 1.00
OverallCond, Min pre normalization: 9.00, 1.00
TotalBsmtSF, Min pre normalization: 6110.00, 0.00
FullBath, Min pre normalization: 3.00, 0.00
HalfBath, Min pre normalization: 2.00, 0.00
BedroomAbvGr, Min pre normalization: 8.00, 0.00
TotRmsAbvGrd, Min pre normalization: 14.00, 2.00
Fireplaces, Min pre normalization: 3.00, 0.00
GarageArea, Min pre normalization: 1418.00, 0.00

After normalization:
LotArea, Min post normalization: 20.52, -0.92
OverallQual, Min post normalization: 2.82, -3.69
OverallCond, Min post normalization: 3.08, -4.11
TotalBsmtSF, Min post normalization: 11.52, -2.41
FullBath, Min post normalization: 2.61, -2.84
HalfBath, Min post normalization: 3.22, -0.76
BedroomAbvGr, Min post normalization: 6.29, -3.51
TotRmsAbvGrd, Min post normalization: 4.60, -2.78
Fireplaces, Min post normalization: 3.70, -0.95
GarageArea, Min post normalization: 4.42, -2.2

### W and B parameter  
column of w = node

In [7]:
#3 layers, layer1 nodes = 4 , layer2 nodes = 4 
#  input layers  rand(input,nodes)  10,4
#  layers 1      rand(layer1 nodes,layer2 nodes)  4,4
#  layers 2      rand(layer2 nodes,output nodes)  4,1
W1_tmp = np.random.rand(10,4)
b1_tmp = np.random.rand(4)
# print(W1_tmp, b1_tmp)

W2_tmp = np.random.rand(4,4)
b2_tmp = np.random.rand(4)

W3_tmp = np.random.rand(4,1)
b3_tmp = np.random.rand(1)

print(W2_tmp)


[[0.6885382  0.39358039 0.69086242 0.53001451]
 [0.47726293 0.23188304 0.68298484 0.73569625]
 [0.92143722 0.24921995 0.22551605 0.1250575 ]
 [0.77709686 0.90523287 0.75811296 0.23139107]]


## Activation Function

In [8]:
#binary  ifx<0 return 0,  ifx>0 return 1
def binaryStep(x):
    return np.heaviside(x,1)

#sigmoid
def g(z):
    return 1 / (1 + np.exp(-z))

#linear
def linear(z):
    return z

#Tanh (nonlinear activation function, outputs between -1 and 1)
def tanh(z):
    return np.tanh(z)

#ReLu --> Return 0 if the input is negative otherwise return the input as it is.
def relu(x):
    return max(0.0,x)

#Leaky ReLU
def Leaky_relu(x):
    if x>0 :
        return x
    else :
        return 0.01*x

### Numpy Model (Forward Prop in NumPy)

In [9]:
def my_dense(a_in, W, b):
    units = W.shape[1]  #3
    a_out = np.zeros(units) #(0,0,0)
    for j in range(units):     #for j in range(3):          
        w = W[:,j]             # w = W[0:the end,0] --> [-8.93,  0.29, 12.9 ]  
                               # w = W[0:the end,1] --> [-0.1,  -7.32, 10.81]
        z = np.dot(w, a_in) + b[j]         #(  x(row) * w(column) + .....)+ b
        a_out[j] = g(z)               
    return(a_out)

### Number of layer

In [10]:
def my_sequential(X, W1, b1, W2, b2, W3, b3):
    a1 = my_dense(X,  W1, b1)
    a2 = my_dense(a1, W2, b2)
    a3 = my_dense(a2, W3, b3)
    return(a3)

### Predictions

In [11]:
def my_predict(X, W1, b1, W2, b2, W3, b3):
    m = X.shape[0]  
    p = np.zeros((m,1)) 
    for i in range(m): 
        p[i] = my_sequential(X[i], W1, b1, W2, b2, W3, b3)  
    return(p)

In [12]:
X_tst = np.array([
[ 8450,     7,     5,   856,     2,     1,     3,     8,     0,548],
[ 9600,     6,     8,  1262,     2,     0,     3,     6,     1,460],
[11250,     7,     5,   920,     2,     1,     3,     6,     1,608]
])

# negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp)

In [13]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)): #predictions = X_tst
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [1.]
 [1.]]


In [14]:
print(predictions)

[[0.75250087]
 [0.75491448]
 [0.75609755]]
